In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
%matplotlib inline
from IPython.core.display import clear_output, display
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import MinMaxScaler

Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Load Dataset

In [4]:
filename = '/content/drive/MyDrive/training/train_finedust_real_gen_1143.csv'

In [5]:
df_data = pd.read_csv(filename)
df_data

,date,ground_pm,ground_temper,avg_pm,avg_temper,wind,humid,max_height
0,NaN,5.769822,0.691519,1,1.1,0.3,90,39.5
1,NaN,3.918659,0.944229,1,1.1,0.3,90,17.2
2,NaN,-0.913783,0.676092,1,1.1,0.3,90,26.3
3,NaN,0.415542,1.342938,1,1.1,0.3,90,39.8
4,NaN,-1.717684,1.455120,1,1.1,0.3,90,24.4
...,...,...,...,...,...,...,...,...
1138,NaN,26.437096,16.725000,34,16.0,2.5,74,37.5
1139,NaN,24.518182,16.700000,34,16.0,2.5,74,22.5
1140,NaN,21.774078,16.574031,34,16.0,2.5,74,5.0
1141,NaN,17.890490,15.796070,34,16.0,2.5,74,7.5


Training Data

In [ ]:
df_train = df_data[["ground_pm", "ground_temper", "avg_pm", "avg_temper", "wind", "humid"]]

In [ ]:
df_train

In [ ]:
x_scaler = MinMaxScaler()
df_train_scaled = x_scaler.fit_transform(df_train)
print("Min:", np.min(df_train_scaled))
print("Max:", np.max(df_train_scaled))

In [ ]:
df_train_scaled

Target Data

In [ ]:
df_target = df_data[["max_height"]]

In [ ]:
df_target

In [ ]:
y_scaler = MinMaxScaler()
df_target_scaled = y_scaler.fit_transform(df_target)
print("Min:", np.min(df_target_scaled))
print("Max:", np.max(df_target_scaled))

In [ ]:
df_target_scaled

Shape of train and target dataset

In [ ]:
print(df_train_scaled.shape)
print(df_target_scaled.shape)

(1143, 6)
(1143, 1)


Placeholder

In [ ]:
X = tf.placeholder(tf.float32, [None, 6], name='train_data')
Y = tf.placeholder(tf.float32, [None, 1], name='target_data')

Define add_layer function
*   Add one more layer and return the output of this layer

In [ ]:
def add_layer(inputs, in_size, out_size, activation_function=None):
  Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='w')
  biases = tf.Variable(tf.random_normal([1, out_size]), name='b')
  Wx_plus_b = tf.matmul(inputs, Weights) + biases
  
  if activation_function is None:
    outputs = Wx_plus_b
  else:
    outputs = activation_function(Wx_plus_b)
  
  return outputs

Build a model

In [ ]:
num_of_node = 100
# add hidden layer
layer_1 = add_layer(X, 6, num_of_node, activation_function=tf.nn.sigmoid)
layer_2 = add_layer(layer_1, num_of_node, num_of_node, activation_function=tf.nn.sigmoid)
layer_3 = add_layer(layer_2, num_of_node, num_of_node, activation_function=tf.nn.sigmoid)
layer_4 = add_layer(layer_3, num_of_node, num_of_node, activation_function=tf.nn.sigmoid)
layer_5 = add_layer(layer_4, num_of_node, num_of_node, activation_function=tf.nn.sigmoid)
layer_6 = add_layer(layer_5, num_of_node, num_of_node, activation_function=tf.nn.sigmoid)

output = add_layer(layer_6, num_of_node, 1, activation_function=tf.nn.sigmoid)

Cost function
*   Mean Squared Error

Optimizer
*   Adam

In [ ]:
cost = tf.reduce_mean(tf.square(Y - output))
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [ ]:
date = np.array(range(len(df_train_scaled)))
date

array([   0,    1,    2, ..., 1140, 1141, 1142])

Training the model

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(date, df_target_scaled, 'b-', lw=6)
plt.xlabel("Time Date")
plt.ylabel("Hight")

saver = tf.train.Saver()
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for step in range(10001):
    cost_val, _ = sess.run([cost, train], feed_dict={X:df_train_scaled, Y:df_target_scaled})
    if step % 100 == 0:
      try:
        lines[0].remove()
      except Exception:
        pass
      
      prediction_value = sess.run(output, feed_dict={X: df_train_scaled})
      lines = ax.plot(date, prediction_value, 'r-', lw=3)

      time.sleep(0.5)
      clear_output()
      display(fig)
      plt.pause(1)

      print('step : ', step, ' cost : ', cost_val)

  saver.save(sess, "checkpoint/fine_dust_dnn_checkpoint.ckpt")  

plt.close()

Testing the model

In [ ]:
filename = '/content/drive/MyDrive/testing/test_finedust_real_gen_124.csv'

In [ ]:
df_test = pd.read_csv(filename)
df_test

In [ ]:
df_test_input = df_data[["ground_pm", "ground_temper", "avg_pm", "avg_temper", "wind", "humid"]]
df_test_target = df_data[["max_height"]]

In [ ]:
x_test_scaler = MinMaxScaler()
df_test_input_scaled = x_test_scaler.fit_transform(df_test_input)

y_test_scaler = MinMaxScaler()
df_test_target_scaled = y_test_scaler.fit_transform(df_test_target)

In [ ]:
df_test_target_scaled

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(date, df_target_scaled, 'b-', lw=6)
plt.xlabel("Time Date")
plt.ylabel("Hight")

saver = tf.train.Saver()
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  saver.restore(sess, "checkpoint/fine_dust_dnn_checkpoint.ckpt")

  prediction_value = sess.run(output, feed_dict={X: df_test_input_scaled})
  lines = ax.plot(date, prediction_value, 'r-', lw=3)

  time.sleep(0.5)
  clear_output()
  display(fig)
  plt.pause(1)


  print("Cost:", cost.eval({X: df_test_input_scaled, Y: df_test_target_scaled}))